In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# predict_future_sales

In [ ]:
#import function tools
import pandas as pd 
import numpy as np
from pandas_profiling import ProfileReport as pp
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error as mse #to idetify the effict of LSTM
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales//item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales//items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales//shops.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales//test.csv')
submission = pd.read_csv('../input/competitive-data-science-predict-future-sales//sample_submission.csv')

In [ ]:
#show the date type of df
sales.dtypes

In [ ]:
#show the head of dataframe
sales.head(5)

In [ ]:
#drop the wrong data
from datetime import datetime
sales=sales[(sales['item_cnt_day']>0) & (sales['item_price']>0)]
sales['date']=pd.to_datetime(sales['date'])

In [ ]:
#explore the data
sales.describe()

In [ ]:
sales.shape

In [ ]:
item_cat.info()

In [ ]:
item_cat.head()

In [ ]:
item_cat.shape

In [ ]:
test.info()

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
submission.head()

*** preprocession of the date **


In [ ]:
sales_group=sales.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].sum()

In [ ]:
sales_group.head(20)

In [ ]:
sales_month_group = sales.groupby('date_block_num')['item_cnt_day'].sum()

In [ ]:
sales_month_group.head()

In [ ]:
# plot the month_sales_cnt
plt.figure(figsize=(16,8))
plt.plot(sales_month_group)
plt.title('month_item_cnt')
plt.xlabel('month')
plt.ylabel('item_cnt_sum')
plt.show()

In [ ]:
#sales data pivot_show
sales_stack=sales.pivot_table(
    index=['shop_id','item_id'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc='sum').reset_index()

In [ ]:
sales_stack

In [ ]:
sales_stack.shape

In [ ]:
# merge to select the shop and item in test data for predict
sales_month_full = pd.merge(test,sales_stack,on = ['shop_id', 'item_id'],how = 'left')
sales_month_full.head()

from the data above data,the NaN means these is not item sales at this month.so we should handle it

In [ ]:
sales_month_full.fillna(0,inplace=True)

In [ ]:
sales_month_full.head()

In [ ]:
sales_month_full=sales_month_full.drop(['ID', 'shop_id', 'item_id'], axis=1)

In [ ]:
sales_month_full

In [ ]:
sales_month_full.shape

In [ ]:
shop_item_train, sales_cnt_train = sales_month_full.values[:,:-2], sales_month_full.values[:, -2:-1].ravel()
shop_item_valid, sales_cnt_valid = sales_month_full.values[:,1:-1], sales_month_full.values[:, -1:].ravel()
shop_item_test = sales_month_full.values[:, 2:]

In [ ]:
shop_item_train

In [ ]:
sales_cnt_train

In [ ]:
shop_item_valid

In [ ]:
sales_cnt_valid

In [ ]:
shop_item_train.shape

In [ ]:
shop_item_test

# Ready to prepare the train_data and test_data
# Let's play the LSTM model and predict!


In [ ]:
#Model function tools
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from sklearn.metrics import mean_squared_error

In [ ]:
# set the model structure
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Reshape(input_shape=(32,), target_shape=(32, 1,)),
    tf.keras.layers.LSTM(units=32, input_shape=(32, 1)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1)
])

In [ ]:
# get the mse_value to identify the effect of model
lstm_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])

In [ ]:
lstm_model.summary()

In [ ]:
#get the process of train model
model_his = lstm_model.fit(shop_item_train, sales_cnt_train, batch_size = 128,validation_split=0.1,epochs = 5)

In [ ]:
# check the model train effect
plt.plot(np.arange(len(model_his.history['mean_squared_error'])), model_his.history['mean_squared_error'])
plt.plot(np.arange(len(model_his.history['val_mean_squared_error'])), model_his.history['val_mean_squared_error'])
plt.legend(['mse', 'val_mse'])

In [ ]:
# modify the model by valid data pre mse

sales_cnt_valid_pre = lstm_model.predict(shop_item_valid)
print('LSTM RMSE =', mean_squared_error(sales_cnt_valid, sales_cnt_valid_pre, squared=False))

#  Predict the last test shop item_cnt

In [ ]:
# predict the test_label data
submission['shop_item_lstm_pre'] = lstm_model.predict(shop_item_test)

In [ ]:
submission

In [ ]:
submission = submission[['ID','shop_item_lstm_pre']]
submission.columns=['ID','item_cnt_month']

In [ ]:
submission

In [ ]:
submission.to_csv(f'submission_result.csv',index=False)